In [ ]:
# -*- coding: utf-8 -*-
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tqdm import tqdm
from korpat_tokenizer import Tokenizer  # Ensure this file is available
!nvidia-smi

2024-11-29 11:30:02.433441: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


Fri Nov 29 11:30:03 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.28.03              Driver Version: 560.28.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A5000               Off |   00000000:01:00.0 Off |                  Off |
| 31%   53C    P0             87W /  230W |     239MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
xlsx_path_train = '/home/billy/rd/dataset/DS학술제-모델링경진대회_Train.xlsx'
xlsx_path_test = '/home/billy/rd/dataset/DS학술제-모델링경진대회_Valid.xlsx'

config_path = "./pretrained/korpat_bert_config.json"  # KorPatBert 설정 파일 경로
vocab_path = "./pretrained/korpat_vocab.txt"         # KorPatTokenizer 어휘 사전 파일 경로
checkpoint_path = "./pretrained/model.ckpt-381250"        # KorPatBert 모델 체크포인트 경로
pretrained_model_dir = "./pretrained/"                    # KorPatBert 모델 디렉토리
save_model_path = "./korpat_bert_test_model.h5"           # 학습된 모델을 저장할 경로

# 학습 파라미터
MAX_SEQ_LEN = 256
BATCH_SIZE = 8
EPOCHS = 5
LR = 0.00003

In [ ]:
from sklearn.model_selection import train_test_split


def load_data(data_path: str) -> pd.DataFrame:
    df = pd.read_excel(data_path,
                       usecols=['발명의 명칭', '요약', '메인IPC2', '대표청구항'])
    df['input'] = df.apply(
        lambda row: f"{row['발명의 명칭']}\n\n{row['요약']}", axis=1)
    df = df.drop(columns=['발명의 명칭', '요약', '대표청구항'])
    df.columns = ['메인IPC2', '대표청구항']
    return df


# 토크나이저 설정
tokenizer = Tokenizer(vocab_path=vocab_path, cased=True)

# 엑셀 파일에서 데이터 로드
df_train = load_data(xlsx_path_train)
df_test = load_data(xlsx_path_test)

# 레이블 인코딩
label_encoder = LabelEncoder()
df_train['label'] = label_encoder.fit_transform(df_train['메인IPC2'])

# 레이블 인코딩 결과 출력
print("레이블 인코딩 결과:")
for label, index in zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)):
    print(f"{label}: {index}")

# 테스트 데이터의 보이지 않는 레이블 처리
unseen_labels = set(df_test['메인IPC2']) - set(label_encoder.classes_)
if unseen_labels:
    print("주의: 테스트 데이터셋에 훈련 데이터셋에 없는 레이블이 있습니다. 해당 샘플을 제거합니다.")
    print(f"제거될 레이블: {unseen_labels}")
    df_test = df_test[~df_test['메인IPC2'].isin(
        unseen_labels)].reset_index(drop=True)

# 테스트 레이블 변환
df_test['label'] = label_encoder.transform(df_test['메인IPC2'])

# 학습 데이터를 학습 세트와 검증 세트로 분할
train_texts_full = df_train['대표청구항']
train_labels_full = df_train['label']

train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts_full, train_labels_full, test_size=0.1, random_state=42, stratify=train_labels_full
)

test_texts = df_test['대표청구항']
test_labels = df_test['label']

# 학습, 검증 및 테스트 데이터를 위한 DataFrame 생성
train_data = pd.DataFrame({'sentence': train_texts, 'label': train_labels})
val_data = pd.DataFrame({'sentence': val_texts, 'label': val_labels})
test_data = pd.DataFrame({'sentence': test_texts, 'label': test_labels})

# 클래스 수
num_classes = len(label_encoder.classes_)
print("레이블 클래스 수:", num_classes)

레이블 인코딩 결과:
G06F: 0
G06Q: 1
G16H: 2
레이블 클래스 수: 3


In [ ]:
from tensorflow.keras.utils import to_categorical


def preprocessing_dataset(dataset, num_classes):
    tokens, indices, labels = [], [], []

    for label, sentence in tqdm(zip(dataset['label'], dataset['sentence']), desc="데이터 전처리 진행중"):
        tokens.append(tokenizer.tokenize(sentence))
        ids, _ = tokenizer.encode(sentence, max_len=MAX_SEQ_LEN)
        indices.append(ids)
        # 레이블을 원-핫 인코딩 벡터로 변환
        labels.append(to_categorical(label, num_classes=num_classes))

    x_data = np.array(indices)
    y_data = np.array(labels)
    print("===> 전처리 결과 출력:")
    print("===> tokens sample:", tokens[0])
    print("===> indices sample:", indices[0])
    print("===> x_data shape:", x_data.shape)
    print("===> y_data shape:", y_data.shape)

    return tokens, x_data, y_data


print("===> 학습데이터 전처리 시작")
train_tokens, train_x, train_y = preprocessing_dataset(train_data, num_classes)

print("\n===> 검증데이터 전처리 시작")
val_tokens, val_x, val_y = preprocessing_dataset(val_data, num_classes)

print("\n===> 테스트데이터 전처리 시작")
test_tokens, test_x, test_y = preprocessing_dataset(test_data, num_classes)

===> 학습데이터 전처리 시작


데이터 전처리 진행중: 2163it [00:13, 161.47it/s]


===> 전처리 결과 출력:
===> tokens sample: ['[CLS]', '[', '청구', '항', '1', ']', '사용', '자', '의', '모바일', '단말기', '및', '상점', '의', 'POS', '단말기', '와', '연동', '된', '커', '##머스', '서버', '에서', '모바일', '커', '##머스', '서비스', '를', '제공', '하', '는', '방법', '에', '있', '어서', ',', '통신', '네트워크', '를', '통하', '여', '모바일', '커', '##머스', '에', '관련', '된', '[UNK]', '데이터', '를', '수집', '하', '는', '단계', ';', '수집', '된', '[UNK]', '데이터', '로부터', '만족', '도', '및', '영향', '##력', '을', '분석', '하', '여', '상점', '또는', '상품', '에', '대한', '랭킹', '정보', '를', '생성', '하', '는', '단계', ';', '상기', '랭킹', '정보', '를', '사용', '자', '의', '모바일', '단말기', '로', '제공', '하', '는', '단계', ';', '상기', '사용', '자', '의', '모바일', '단말기', '로부터', '구매', '하', '고자', '하', '는', '상품', '및', '상점', '정보', '를', '포함', '하', '는', '구매', '요청', '을', '수신', '하', '는', '단계', ';', '상기', '구매', '요청', '에', '따라', '모바일', '상품권', '또는', '주문', '정보', '를', '사용', '자', '의', '모바일', '단말기', '로', '전송', '하', '는', '단계', ';', '및', '상기', '모바일', '상품권', '또는', '주문', '정보', '가', '사용', '된', '상점', '을', '대상', '으로', '정산', '을', '진행', '하', '는', '

데이터 전처리 진행중: 241it [00:01, 151.40it/s]


===> 전처리 결과 출력:
===> tokens sample: ['[CLS]', '[', '청구', '항', '1', ']', '회원', '사', '로부터', '회원', '아이디', ',', '사업자', '정보', ',', '매출', ',', '직원', '수', ',', '임', '##금', '정보', '를', '주기', '적', '으로', '수신', '하', '여', '회원', '사', '정보', 'DB', '에', '저장', '및', '관리', '하', '는', '[UNK]', '데이터', '관리', '부', ';', '회원', '사', '로부터', '노', '##무', '분석', '요청', '을', '수신', '하', '는', '노', '##무', '분석', '접', '##수부', ';', '상기', '회원', '사', '정보', 'DB', '에', '저장', '된', '사업자', '정보', '중', '업', '##태', '와', '종목', ',', '매출', '증감', '정도', ',', '직원', '수', '변화', ',', '임', '##금', '상승', '정도', '를', '기계', '학습', '(', 'Machine', 'L', '##ear', '##ning', ')', '하', '여', '업', '##태', '와', '종목', ',', '매출', ',', '직원', '수', ',', '임', '##금', '정보', '에', '따라', '신규', '직원', '채용', '을', '언제', '하', '면', '되', '는지', '또는', '기존', '직원', '의', '임', '##금', '을', '어느', '정도', '로', '인상', '하', '면', '되', '는지', '를', '추론', '하', '는', '노', '##무', '관리', '모델', '을', '생성', '하', '는', '노', '##무', '관리', '모델', '생', '성', '##부', '-', '입력', '변수', '에', '해당', '하', '는', '업', '##태'

데이터 전처리 진행중: 602it [00:04, 137.72it/s]

===> 전처리 결과 출력:
===> tokens sample: ['[CLS]', '[', '청구', '항', '2', ']', '메인', '서버', '(', '20', ')', '는', '저장', '장치', '(', '30', ')', '에', '저장', '시킨', '[UNK]', '데', '이타', '를', '기초', '로', '정신', '및', '심리', '분석', '선정', '단어', ',', '색상', ',', '그림', ',', '특정', '행동', '등', '을', '샘플링', '하', '여', '저장', '하', '고', ',', '메인', '서버', '에서', '환자', '의', '투', '##병', '관련', '기록', '인', '종이', '시트', '나', '온라인', '상', '의', '플랫폼', '에서', '작성', '하', '거나', '오프라인', '상', '에서', '기록', '한', '워드', '파일', '이나', ',', '기록', '사진', '이나', '녹음', '파일', '이나', '영상', '파일', '등', '으로', '저장', '하', '고', ',', '저장', '파일', '을', '메인', '서버', '가', '텍스트', '파일', ',', '녹음', '파일', ',', '그림', '파일', '등', '으로', '구분', '하', '여', '변환', '한', '것', '을', '기초', '로', '우울증', '에', '해당', '하', '는', '인식', '단어', '나', '인식', '색상', '이나', '인식', '영상', '을', '차례', '로', '검색', '하', '는', '검색', '모듈', '(', '50', ')', ';', '검색', '모듈', '(', '50', ')', '에서', '검색', '단어', '나', '색상', '이나', '영상', '이', '있', '는지', '를', '순차', '선택', '하', '는', '멀티', '##플', '레', '##서', '(', '60', ')', ';',

In [ ]:
# 분류 모델 네트워크 정의
import bert

input_ids = keras.layers.Input(shape=(MAX_SEQ_LEN,), dtype='int32')

# BERT 언어모델 레이어 생성
bert_params = bert.params_from_pretrained_ckpt(pretrained_model_dir)
l_bert = bert.BertModelLayer.from_params(bert_params, name="bert")

# 입력레이어와 BERT 레이어 연결
bert_output = l_bert(input_ids)
print("bert shape", bert_output.shape)

2024-11-29 11:30:23.590946: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2024-11-29 11:30:23.693626: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-11-29 11:30:23.696757: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA RTX A5000 computeCapability: 8.6
coreClock: 1.695GHz coreCount: 64 deviceMemorySize: 23.58GiB deviceMemoryBandwidth: 715.34GiB/s
2024-11-29 11:30:23.696770: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2024-11-29 11:30:23.699129: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2024-11-29 11:30:23.699162: I tensorflow/stream_executor/platform/

bert shape (None, 256, 768)


In [6]:

# BERT 출력 중 [CLS] 토큰 정보를 추출
cls_out = keras.layers.Lambda(lambda seq: seq[:, 0, :])(bert_output)

# 최종 분류를 위한 레이어
outputs = Dense(units=num_classes, activation='softmax')(cls_out)

# 모델 빌드
model = keras.Model(inputs=input_ids, outputs=outputs)
model.build(input_shape=(None, MAX_SEQ_LEN))
model

In [ ]:

# BERT 언어모델의 초기 가중치 로드
bert.load_stock_weights(l_bert, checkpoint_path)

# 모델 컴파일
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LR),
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)
model.summary()

Done loading 196 BERT weights from: ./pretrained/model.ckpt-381250 into <bert.model.BertModelLayer object at 0x7f045bdb04f0> (prefix:bert). Count of weights not found in the checkpoint was: [0]. Count of weights with mismatched shape: [0]
Unused weights from checkpoint: 
	bert/embeddings/LayerNorm/beta/adam_m
	bert/embeddings/LayerNorm/beta/adam_v
	bert/embeddings/LayerNorm/gamma/adam_m
	bert/embeddings/LayerNorm/gamma/adam_v
	bert/embeddings/position_embeddings/adam_m
	bert/embeddings/position_embeddings/adam_v
	bert/embeddings/token_type_embeddings
	bert/embeddings/token_type_embeddings/adam_m
	bert/embeddings/token_type_embeddings/adam_v
	bert/embeddings/word_embeddings/adam_m
	bert/embeddings/word_embeddings/adam_v
	bert/encoder/layer_0/attention/output/LayerNorm/beta/adam_m
	bert/encoder/layer_0/attention/output/LayerNorm/beta/adam_v
	bert/encoder/layer_0/attention/output/LayerNorm/gamma/adam_m
	bert/encoder/layer_0/attention/output/LayerNorm/gamma/adam_v
	bert/encoder/layer_0/att

In [ ]:
# 모델 학습 시작
history = model.fit(
    train_x,
    train_y,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(val_x, val_y)
)

Epoch 1/5


2024-11-29 11:30:25.532085: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2024-11-29 11:30:25.532295: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 3187200000 Hz
2024-11-29 11:30:30.038034: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11


  1/271 [..............................] - ETA: 22:20 - loss: 1.6374 - accuracy: 0.0000e+00

2024-11-29 11:30:30.323175: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2024-11-29 11:30:30.323217: I tensorflow/stream_executor/cuda/cuda_blas.cc:1838] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


271/271 [==============================] - 54s 180ms/step - loss: 0.4714 - accuracy: 0.8155 - val_loss: 0.2874 - val_accuracy: 0.8880
Epoch 2/5
271/271 [==============================] - 48s 177ms/step - loss: 0.2336 - accuracy: 0.9117 - val_loss: 0.3069 - val_accuracy: 0.8880
Epoch 3/5
271/271 [==============================] - 48s 177ms/step - loss: 0.1172 - accuracy: 0.9575 - val_loss: 0.2777 - val_accuracy: 0.8921
Epoch 4/5
271/271 [==============================] - 48s 177ms/step - loss: 0.0465 - accuracy: 0.9857 - val_loss: 0.4117 - val_accuracy: 0.9212
Epoch 5/5
271/271 [==============================] - 48s 177ms/step - loss: 0.0518 - accuracy: 0.9843 - val_loss: 0.3237 - val_accuracy: 0.9170


In [ ]:
# 모델 저장 및 테스트 데이터로 평가
model.save(save_model_path)
eval_result = model.evaluate(test_x, test_y)

print('\n===> 평가 결과:')
print("테스트 손실: %.4f" % (eval_result[0]))
print("테스트 정확도: %.4f" % (eval_result[1]))

# 분류 보고서 생성
y_pred_probs = model.predict(test_x)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(test_y, axis=1)

# 분류 리포트 출력
print('\n분류 리포트:')
print(classification_report(y_true, y_pred,
      target_names=label_encoder.classes_, digits=4))

19/19 [==============================] - 3s 158ms/step - loss: 0.5266 - accuracy: 0.8721

===> 평가 결과:
테스트 손실: 0.5266
테스트 정확도: 0.8721

분류 리포트:
              precision    recall  f1-score   support

        G06F     0.7170    0.7677    0.7415        99
        G06Q     0.9226    0.9308    0.9267       448
        G16H     0.7273    0.5818    0.6465        55

    accuracy                         0.8721       602
   macro avg     0.7889    0.7601    0.7715       602
weighted avg     0.8709    0.8721    0.8706       602

